### Instalación de todos los paquetes necesarios

In [2]:
%pip install llama-index-agent-openai
%pip install llama-index
%pip install azure-search-documents===11.4.0
%pip install llama-index-embeddings-azure-openai
%pip install llama-index-llms-azure-openai
%pip install llama-index-vector-stores-azureaisearch
%pip install --q crewai
%pip install --q -U duckduckgo-search
%pip install --q langchain_openai
%pip install openai
%pip install 'crewai[tools]'

%pip install scrapegraphai
%pip install nest_asyncio
%pip install playwright
!playwright install

  Using cached azure_search_documents-11.4.0-py3-none-any.whl.metadata (22 kB)
Using cached azure_search_documents-11.4.0-py3-none-any.whl (283 kB)
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.5.1
    Uninstalling azure-search-documents-11.5.1:
      Successfully uninstalled azure-search-documents-11.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-azureaisearch 0.2.2 requires azure-search-documents<12.0.0,>=11.5.1, but you have azure-search-documents 11.4.0 which is incompatible.
  Using cached azure_search_documents-11.5.1-py3-none-any.whl.metadata (23 kB)
Using cached azure_search_documents-11.5.1-py3-none-any.whl (297 kB)
  Attempting uninstall: azure-search-documents
    Found existing installation: azure-search-documents 11.4.0
    Uninstalling azure-search-documents

### Importar todos los paquetes y módulos necesarios

In [3]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.llm import LLM
import openai
import requests
import os

import json
import pandas as pd

from llama_index.core import SimpleDirectoryReader, Document
from llama_index.core import VectorStoreIndex

from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings

from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement
from llama_index.vector_stores.azureaisearch import (
    IndexManagement,
    MetadataIndexFieldType,
)
import nest_asyncio
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter

from crewai_tools import BaseTool
from pydantic import BaseModel
from typing import Any

### Configurar API

In [ ]:

OPENAI_API_KEY = "xxxxxxxxxxxxxxxxxxxxxxxx"

# Establecemos el siguiente LLM para CREW-AI.
# Para crear el índice, hacemos uso de otro.

azure_llm = LLM(api_key=OPENAI_API_KEY,
                base_url="https://openaicore-sweden.openai.azure.com",
                api_version="2024-02-15-preview",
                model="azure/gpt-4-turbo",
                temperature = 0.6)

print("Azure OpenAI LLM values are:")
print(azure_llm.api_key)
print(azure_llm.base_url)
print(azure_llm.api_version)
print(azure_llm.model)
print(azure_llm.temperature)


Azure OpenAI LLM values are:
91b1e4640595414eb2cdab721eb0bd5d
https://openaicore-sweden.openai.azure.com
2024-02-15-preview
azure/gpt-4-turbo
0.6


In [5]:
azure_llm

# Scrapping de las páginas de la UAL correspondientes

In [6]:
import nest_asyncio
nest_asyncio.apply()

from langchain_openai import AzureChatOpenAI
from typing import List
from scrapegraphai.graphs import SmartScraperMultiGraph

from pydantic import BaseModel, Field

class Normativa(BaseModel):
    name: str = Field(description="The name of the normativa")
    link: str = Field(description="The url of the normativa")

class Normativas(BaseModel):
    normativas: List[Normativa]


llm = AzureChatOpenAI(deployment_name="gpt-4-turbo",
                      openai_api_version="2024-02-15-preview",
                      openai_api_key=OPENAI_API_KEY,
                      azure_endpoint="https://openaicore-sweden.openai.azure.com")
model_tokens_count = 10000

graph_config = {
    "llm": {
        "model_instance": llm,
        "model_tokens": model_tokens_count,
        "temperature": 0,
        #"api_version": "2024-02-15-preview",
        #"api_key": OPENAI_API_KEY,
        #"model": "azure_openai/gpt-3.5-turbo",
        #"azure_endpoint": "https://openaicore-sweden.openai.azure.com"
    },
    "verbose": True,
    "loader_kwargs": {
        "slow_mo": 3000
    },
}


smart_scraper_graph = SmartScraperMultiGraph(
    prompt="List all links under Normativa y Documentación with title and download link.",
    # also accepts a string with the already downloaded HTML code
    source=["https://www.ual.es/universidad/serviciosgenerales/araties/normativa", "https://www.ual.es/estudios/gestionesacademicas/secretaria/automatricula/normativa"],
    config=graph_config,
    schema=Normativas
)

result = smart_scraper_graph.run()
print(result)

--- Executing GraphIterator Node with batchsize 16 ---
processing graph instances:   0%|          | 0/2 [00:00<?, ?it/s]--- Executing Fetch Node ---
--- Executing Fetch Node ---
--- (Fetching HTML from: https://www.ual.es/estudios/gestionesacademicas/secretaria/automatricula/normativa) ---
--- (Fetching HTML from: https://www.ual.es/universidad/serviciosgenerales/araties/normativa) ---
--- Executing ParseNode Node ---
--- Executing ParseNode Node ---
--- Executing GenerateAnswer Node ---
--- Executing GenerateAnswer Node ---
processing graph instances: 100%|██████████| 2/2 [00:37<00:00, 18.60s/it]
--- Executing MergeAnswers Node ---


{'normativas': [{'name': 'Reglamento de Administración Electrónica de la UAL', 'link': 'https://www.juntadeandalucia.es/boja/2021/40/BOJA21-040-00012-3337-01_00187259.pdf'}, {'name': 'Normativa de Reconocimiento y Transferencia de Créditos', 'link': 'https://www.ual.es/application/files/6816/4500/3188/Normativa_de_Reconocimiento_y_Transferencia_de_creditos_en_Grado_y_Master_de_la_UAL.pdf'}, {'name': 'Normativa de Permanencia en la Universidad', 'link': 'https://www.ual.es/application/files/2316/2643/4199/Permanencia_Estudiantes_en_la_UAL.pdf'}, {'name': 'Normativa de traslados de expediente', 'link': 'https://www.ual.es/application/files/5516/5528/0650/Procedimiento_Admision_Traslado_de_Expediente.pdf'}, {'name': 'Normativa de Extinción de las Enseñanzas', 'link': 'https://www.ual.es/application/files/2416/5512/3200/Normativa_Extincion_Titulaciones_Oficiales.pdf'}, {'name': 'Reglamento sobre Evaluación del Aprendizaje en la UAL', 'link': 'https://www.ual.es/application/files/5716/2332/

In [7]:
from scrapegraphai.utils import convert_to_json, prettify_exec_info


# ************************************************
# Get graph execution info
# ************************************************

graph_exec_info = smart_scraper_graph.get_execution_info()
print(prettify_exec_info(graph_exec_info))


# Save to json
convert_to_json(result, "dataset_ual", "/content/")

       node_name  total_tokens  prompt_tokens  completion_tokens  \
0  GraphIterator         11456          10506                950   
1   MergeAnswers          2146           1259                887   
2   TOTAL RESULT         13602          11765               1837   

   successful_requests  total_cost_USD  exec_time  
0                    2         0.13356  37.221686  
1                    1         0.03920  37.116808  
2                    3         0.17276  74.338494  


### Obtener y descargar los PDFs

In [8]:

with open('dataset_ual.json', 'r') as f:
       data = json.load(f)

df = pd.DataFrame(data['normativas'])

df.head()

def download_pdf(name, url, folder_path):

  os.makedirs(folder_path, exist_ok=True)
  file_path = os.path.join(folder_path, name + '.pdf')

  try:

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(file_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"PDF downloaded and saved to: {file_path}")

  except requests.exceptions.RequestException as e:
    print(f"Error downloading PDF: {e}")

for index, row in df.iterrows():
  download_pdf(row['name'], row['link'], 'pdfs')

PDF downloaded and saved to: pdfs/Reglamento de Administración Electrónica de la UAL.pdf
PDF downloaded and saved to: pdfs/Normativa de Reconocimiento y Transferencia de Créditos.pdf
PDF downloaded and saved to: pdfs/Normativa de Permanencia en la Universidad.pdf
PDF downloaded and saved to: pdfs/Normativa de traslados de expediente.pdf
PDF downloaded and saved to: pdfs/Normativa de Extinción de las Enseñanzas.pdf
PDF downloaded and saved to: pdfs/Reglamento sobre Evaluación del Aprendizaje en la UAL.pdf
PDF downloaded and saved to: pdfs/Reglamento de calificaciones.pdf
PDF downloaded and saved to: pdfs/Normativa de organización y evaluación de los trabajos fin de estudios.pdf
PDF downloaded and saved to: pdfs/Normativa de estudios oficiales de doctorado de la UAL.pdf
PDF downloaded and saved to: pdfs/Normativa para la admisión de estudiantes visitantes.pdf
PDF downloaded and saved to: pdfs/Matrícula oficial 2024-25 (Grado y Máster).pdf
PDF downloaded and saved to: pdfs/Matrícula ofici

## ❌ NO EJECUTAR ❌

In [9]:
### IMPORTANTE ###

'''
# No ejecutar. Esta es otra forma de obtener el índice.
# No obstante, es menos preciso que el que hay en la siguiente celda.
'''

#ual_documents = SimpleDirectoryReader("pdfs").load_data()

#ual_index_test = VectorStoreIndex.from_documents(ual_documents)

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

## Configuración de Modelos LLM y Embeddings

In [10]:
azure_endpoint = "https://openaicore-sweden.openai.azure.com/"
api_version = "2023-07-01-preview"

# Este LLM es necesario para poder hacer una llamada a esta función más adelante:
# index.as_query_engine(similarity_top_k=3)

llm = AzureOpenAI(
    model="gpt-4-turbo",
    deployment_name="gpt-4-turbo",
    api_key=OPENAI_API_KEY,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)


# Este es el modelo utilizado para obtener los embeddings.

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-large",
    deployment_name="text-embedding-3-large",
    api_key=OPENAI_API_KEY,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

Settings.llm = llm
Settings.embed_model = embed_model

## Configuración de Índice Vectorial y Búsqueda por Similitud en Azure Cognitive Search

In [ ]:
nest_asyncio.apply()

INDEX_NAME = "aelectronica-legal-ual-machacateclados"

# Configuración del Cliente de búsqueda de Azure
credential = AzureKeyCredential("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
index_client = SearchIndexClient(endpoint="https://aisearchcore-indesia.search.windows.net", credential=credential)

vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=INDEX_NAME,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="text",
    embedding_field_key="embedding",
    embedding_dimensionality=3072,
    metadata_string_field_key="metadata",
    doc_id_field_key="parent_id",
    language_analyzer="es.lucene",
    vector_algorithm_type="exhaustiveKnn",
)

# Configurar el text splitter
text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

# Cargar los documentos
storage_context = StorageContext.from_defaults(vector_store=vector_store)


reader = SimpleDirectoryReader(input_dir="pdfs")
llama_documents = reader.load_data()

# Crear el índice
index = VectorStoreIndex.from_documents(
    llama_documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

## Creación de Motor de Búsqueda basado en similitud con embeddings

In [ ]:
ual_engine = index.as_query_engine(similarity_top_k=3)

## Creación de las herramientas que usará cada Agente (departamento)

In [ ]:
# Creamos las herramientas necesarias para cada departamento (AGENTE)

class UALTool(BaseTool, BaseModel):
    name: str
    description: str
    query_engine: Any

    def _run(self, argument: str) -> str:
        result = self.query_engine.query(argument)
        return result

# Crear instancias de la herramienta para cada tipo de consulta
becas_tool = UALTool(
    name="Becas",
    description=(
        "Proporciona información sobre becas disponibles para estudiantes. "
        "Utiliza una pregunta detallada en texto simple como entrada para la herramienta."
    ),
    query_engine=ual_engine
)

economia_tool = UALTool(
    name="Economía",
    description=(
        "Proporciona información sobre la gestión económica del expediente académico. "
        "Utiliza una pregunta detallada en texto simple como entrada para la herramienta."
    ),
    query_engine=ual_engine
)

certificaciones_tool = UALTool(
    name="Certificaciones",
    description=(
        "Proporciona información sobre actas y certificaciones académicas. "
        "Utiliza una pregunta detallada en texto simple como entrada para la herramienta."
    ),
    query_engine=ual_engine
)

# Metemos todo en un array para que lo tengamos todo junto (comodidad personal)
query_engine_tools = [becas_tool, economia_tool, certificaciones_tool]


## ⚠️Funcionalidad EXTRA.

>  **EJECUTAR ÚNICAMENTE PARA USAR DUCKDUCKGO-SEARCH Y CONVERTIR EL CONTENIDO EN UN MARKDOWN**



In [ ]:
#### IMPORTANTE ####

'''
# NO EJECUTAR A MENOS QUE SE QUIERA USAR DUCKDUCKGO-SEARCH Y CONVERTIR
# EL CONTENIDO EN UN MARKDOWN, SE CREARÍA ESTA FUNCIÓN QUE LUEGO
# SE LE PASARÍA AL AGENTE EN CUESTIÓN.
'''

#!pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()


import os
from langchain.tools import StructuredTool, Tool

# Definir la función que realiza la conversión y el guardado
def save_to_markdown(content: str, filename: str):
    # Convertir el contenido a Markdown (personaliza esto según tus necesidades)
    markdown_content = f"# AI Advancements\n\n{content}"

    # Guardar el contenido en un archivo Markdown
    with open(filename, "w") as f:
        f.write(markdown_content)

    return f"Content saved to {filename}"

# Crear instancia de la herramienta estructurada
save_tool = StructuredTool.from_function(
    func=save_to_markdown,
    name="SaveToMarkdown",
    description="Converts content to Markdown format and saves it to disk.",
    parameters={
        "content": "The content to convert to Markdown format.",
        "filename": "The name of the file where the Markdown content will be saved."
    },
)


### Configurar a todos los agentes

In [ ]:
# Definir a todos los agentes con sus roles y sus objetivos
alumno = Agent(
    role='Alumno',
    goal='Te encargas de hacer preguntas al secretario de la universidad. Hablas y piensas en español.',
    backstory="""Estudias en la universidad y tienes muchas preguntas.""",
    verbose=True,
    allow_delegation=False,
    llm=azure_llm,  #Usando azure OpenAI API
    tools=[]
)

secretario = Agent(
    role='Secretario',
    goal='Responder a la consulta del cliente y en caso de no tener la información necesaria, preguntar al agente correspondiente. Hablas y piensas en español.',
    backstory="""Trabajas en la universidad, proporcionando servicios de atención al cliente. Sabes que
    hay 3 agentes diferentes siendo estos: Jefa de Sección de Becas, Administrador de Actas y Certificaciones Académicas, y
    Responsable de Gestión Económica del Expediente y Documentación. Dependiendo de la consulta del cliente, es importante que le pregunte al
    responsable correspondiente. Una vez ese responsable devuelva su respectiva respuesta, que le comuniques al cliente
    los datos que te han pedido.""",
    verbose=True,
    allow_delegation=True,
    llm=azure_llm,  #Usando azure OpenAI API
    tools=[]  # El secretario no tiene acceso directo a herramientas
)

beca = Agent(
    role='Jefa de Sección de Becas',
    goal='Ayudar a los estudiantes a obtener información y gestionar sus solicitudes de becas. Hablas y piensas en español.',
    backstory="""Si recibes un mensaje del secretario, tienes que proporcionarle toda la información requerida.
    Si no recibes nada, pues no actúas en esta conversación. Encabezas la sección de becas de la universidad, proporcionando asistencia
      para la tramitación de solicitudes y orientación sobre los requisitos de las becas disponibles.
      Tienes un amplio conocimiento sobre los plazos, procedimientos y tipos de ayudas económicas
      que los estudiantes pueden solicitar.""",
    verbose=True,
    allow_delegation=True,
    llm=azure_llm,  #Usando azure OpenAI API
    tools=[query_engine_tools[0]]  # Asigna la herramienta correspondiente a becas
)

certificaciones = Agent(
    role='Administrador de Actas y Certificaciones Académicas',
    goal='Gestionar la emisión de actas y certificaciones académicas para los estudiantes. Hablas y piensas en español.',
    backstory="""Si recibes un mensaje del secretario, tienes que proporcionarle toda la información requerida.
    Si no recibes nada, pues no actúas en esta conversación. Estás a cargo de la gestión de actas y certificados académicos, ayudando a los estudiantes
      a obtener los documentos necesarios para su trayectoria académica. Tienes experiencia en la emisión,
      registro y autenticación de certificaciones y actas.""",
    verbose=True,
    allow_delegation=False,
    llm=azure_llm,  #Usando azure OpenAI API
    tools=[query_engine_tools[2]]  # Asigna la herramienta correspondiente a certificaciones
)

economia = Agent(
    role='Responsable de Gestión Económica del Expediente y Documentación. Hablas y piensas en español.',
    goal='Asistir en la gestión de pagos, cobros y otros aspectos económicos del expediente académico.',
    backstory="""Si recibes un mensaje del secretario, tienes que proporcionarle toda la información requerida.
    Si no recibes nada, pues no actúas en esta conversación. Eres el responsable de la gestión económica del expediente, asegurando que los estudiantes
      estén al día con sus pagos y ayudando con consultas sobre la facturación y los cobros.
      Tienes un conocimiento profundo de los procedimientos financieros y cómo afectan a los estudiantes.""",
    verbose=True,
    allow_delegation=False,
    llm=azure_llm,  #Usando azure OpenAI API
    tools=[query_engine_tools[1]]  # Asigna la herramienta correspondiente a economía
)

### Definir las tareas para los agentes

In [ ]:
#Tarea del agente Alumno
task1 = Task(
  description="""El alumno tiene que hacer la siguiente pregunta exactamente: ¿Qué debo hacer para cambiarme de grupo y cuándo debo hacerlo?""",
  agent=alumno,
  expected_output="""Tu respuesta final tiene que ser una consulta al secretario."""
)


In [ ]:
#Tarea del agente Secretario (debe delegar en los demás y devolver la información en forma de respuesta al alumno)
task2 = Task(
  description="""Responder a la consulta del alumno.""",
  agent=secretario,
  expected_output="""Le devuelves la información requerida al alumno."""
)


### Creación del equipo de agentes

In [ ]:

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[alumno,secretario, beca, certificaciones, economia],
  tasks=[task1, task2],
  verbose=True,
  process=Process.sequential,
)

In [ ]:
crew

### Poner al equipo a trabajar

In [ ]:
result = crew.kickoff()

In [ ]:
print(result)